In [1]:
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io

import pylab
import os
import shutil
import glob
%matplotlib inline

path_dir = './imagenet/train2017'
file_list = os.listdir(path_dir)        # path 에 존재하는 파일 목록 가져오기
file_list.sort()                        # 파일 이름 순서대로 정렬
print("coco데이터수:",len(file_list))

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

dataDir='./imagenet'
dataType='train2017'
annFile='{}/annotations/instances_{}.json'.format(dataDir,dataType)

# initialize COCO api for instance annotations
coco=COCO(annFile)
cats = coco.loadCats(coco.getCatIds())
print(len(cats))#카테고리 수
nms=[cat['name'] for cat in cats]
print('COCO categories: \n{}\n'.format(' '.join(nms)))

image_data={}#key(사진파일이름),value(imgID)
category_data={}#key(카테고리ID),value(카테고리)
find_data={}#key(tuple(imgID)),value(카테고리ID)

def return_imgId(imgName):
    for i in nms:
        #카테고리 단어를 한개 씩 넣어 categoryID추출
        catIds = coco.getCatIds(catNms=[i]);
        category_data[catIds[0]]=i
        
        #print(catIds) #[catIds]
        
        #위의 카테고리 ID를 넣어 imgID추출
        #imgIds = coco.getImgIds(catIds=catIds);
        imgIds = coco.getImgIds(catIds=catIds);
        find_data[tuple(imgIds)]=catIds[0]
        
        
        #imgID를 이용하여 img정보에 대한 API호출
        img = coco.loadImgs(imgIds)
        
        #API형식 리스트속 dictionary
        #리스트속 dictionaey이용해 파일이름 알아내기
     
        #b=1
        for a in img:
            #print(a)
            #file_name를 Key값, id를 Value값으로 dictionary만들기
            #if b<2:
                file_name=(a['file_name'])
                image_id=(a['id'])
                image_data[file_name]=image_id
                #b+=1 
         
    return image_data[imgName]

def findCategory(imgIds):
    img_cate=[]
    for id, cate in find_data.items(): #find_data 아이템을 하나씩 접근해서, id, cate 에 저장
        for x in id:
            if x == imgIds[0]:
                img_cate.append(cate)
    return img_cate
    
#train2017 사진 폴더 리스트
def main():
    print("START")
    for i in file_list:
        #print(i)
        
        #id가안나오는 이미지들은 패스
        try :
            imgIds = coco.getImgIds(imgIds = [return_imgId(i)])  
        except KeyError :
            pass
        
        #카테고리를 이용해 폴더 만들기        
        for y in findCategory(imgIds):
            #path_dir = './imagenet/train2017'
            
            my_category=category_data[y]
            
            #before_path = './imagenet/train2017/사진이름.jpg'
            before_path=path_dir+"/"+i
            
            #train_path = './imagenet/train2017/카테고리'
            train_path=path_dir+"/"+my_category+"/"
            if not os.path.isdir(train_path):
                os.mkdir(train_path)
                shutil.copy2(before_path ,train_path+i)
            else:
                #train_path = './imagenet/test2017/카테고리/사진이름.jpg'
                shutil.copy2(before_path ,train_path+i)
                
    #파일 이동후 삭제
    myExt = '*.jpg' # 확장자            
    files = glob.glob(os.path.join(path_dir, myExt))
    for f in files:
        os.remove(f)
    print("Remove Finish")

    print("ALL Finish")
         
        #사진 테스트 코드
        #imgIds = coco.getImgIds(imgIds = imgIds)
        #img = coco.loadImgs(imgIds)[0]       
        #I = io.imread(img['coco_url'])
        #plt.axis('off')
        #plt.imshow(I)
        #plt.show() 
main()         

coco데이터수: 118287
loading annotations into memory...
Done (t=14.96s)
creating index...
index created!
80
COCO categories: 
person bicycle car motorcycle airplane bus train truck boat traffic light fire hydrant stop sign parking meter bench bird cat dog horse sheep cow elephant bear zebra giraffe backpack umbrella handbag tie suitcase frisbee skis snowboard sports ball kite baseball bat baseball glove skateboard surfboard tennis racket bottle wine glass cup fork knife spoon bowl banana apple sandwich orange broccoli carrot hot dog pizza donut cake chair couch potted plant bed dining table toilet tv laptop mouse remote keyboard cell phone microwave oven toaster sink refrigerator book clock vase scissors teddy bear hair drier toothbrush

START


OSError: [Errno 28] No space left on device